In [1]:
import pandas as pd
from pymongo import MongoClient, GEO2D
from pandas.io.json import json_normalize

In [2]:
client = MongoClient ('localhost', 27017)
db = client.companies
collection = db.companies

In [3]:
# mongoimport --db companies --collection companies --file companies.json
# FILTER {$and: [ {deadpooled_year:null}, {founded_year: {$gt:2003}}, {offices: { $ne: [] } }, {$or: [{category_code:'games_video'},{category_code:'software'}] }]}
# PROJECT {name:1 , category_code:1, number_of_employees:1, founded_year:1, "offices.city":1, "offices.country_code":1, "offices.latitude":1, "offices.longitude":1}
# SORT {number_of_employees:-1}

In [4]:
def query_to_db():
    cursor = collection.find({'$and': 
                        [ {'deadpooled_year':None}, 
                          {'founded_year': {'$gt':2003}},
                          {'offices': { '$ne': [] } },
                          {'$or': [{'category_code':'games_video'},
                                   {'category_code':'software'}] 
                        }]}
                        , {'name':1 , 'category_code':1, 'number_of_employees':1, 'founded_year':1, 
                           "offices.city":1, "offices.country_code":1, "offices.latitude":1, "offices.longitude":1}
                        ).sort('number_of_employees', -1)
    return cursor

In [5]:
def clean_nans(data):
    data= data.dropna(subset=['latitude', 'longitude', 'number_of_employees'])
    return data

In [6]:
def calculate_type(reg):
    if reg['founded_year'] > 2007 and reg['number_of_employees'] < 50:
        return 'Startup'
    else: return 'BigCompany'
    
def get_class(data):
    data['type']= data.apply(calculate_type, axis=1)
    return data

In [7]:
def order_data(data):
    data = data[['name', 'category_code', 'founded_year', 
                  'country_code', 'city', 'latitude', 'longitude', 
                  'number_of_employees']]
    data= data.reset_index(drop=True)
    return data

In [22]:
def get_2d_geo(data):
    data['2Dgeo']= data.apply(lambda reg: [reg['longitude'], reg['latitude']], axis=1)
    return data

In [25]:
#Main

datos_crudo= query_to_db()
df = json_normalize(data= datos_crudo, record_path='offices', 
                    meta=['name', 'category_code', 'number_of_employees', 'founded_year'])
df_clean= clean_nans(df)
df_ordered= order_data(df_clean)
df_final= get_class(df_ordered)
df_geo = get_2d_geo(df_final)
df_geo.to_json('data_normalize_to_plot.json', orient='records', lines= True)

In [26]:
df_geo.head()

,name,category_code,founded_year,country_code,city,latitude,longitude,number_of_employees,type,2Dgeo
0,UCWeb,software,2004,CHN,Guangzhou,23.133754,113.385331,1700,BigCompany,"[113.3853308, 23.1337544]"
1,UCWeb,software,2004,USA,San Francisco,37.781476,-122.401322,1700,BigCompany,"[-122.4013217, 37.7814765]"
2,Veeam Software,software,2006,USA,Columbus,40.088052,-83.088566,1500,BigCompany,"[-83.088566, 40.088052]"
3,HubSpot,software,2006,USA,Cambridge,42.375392,-71.118487,650,BigCompany,"[-71.118487, 42.375392]"
4,Kabam,games_video,2006,USA,San Francisco,37.394456,-122.078531,650,BigCompany,"[-122.0785314, 37.3944561]"
